## 尝试查看井有哪些共同曲线


In [ ]:
"""
读取所有LAS文件并找出共有曲线
"""

import os
from pathlib import Path

import lasio
import pandas as pd

In [ ]:
# ============ 1. 读取所有LAS文件 ============
las_folder = Path("../data/vertical_well_common_las")

# 获取所有LAS文件
input_path = Path(las_folder)
las_files = list(input_path.glob("*.las"))
las_files = list(set(las_files))  # 去重
las_files.sort()

print(f"找到 {len(las_files)} 个 LAS 文件:")
for f in las_files:
    print(f"  - {f.name}")
print("\n" + "=" * 80)

# 存储每个井的曲线信息
well_curves = {}
all_curves = []

for las_file in las_files:
    try:
        las = lasio.read(las_file)
        well_name = las_file.stem
        curves = list(las.keys())

        well_curves[well_name] = curves
        all_curves.extend(curves)

        print(f"✓ {well_name:15s} - {len(curves):3d} 条曲线")

    except Exception as e:
        print(f"✗ 读取 {las_file.name} 失败: {e}")

print("\n" + "=" * 80)

In [ ]:
# ============ 2. 统计曲线出现频率 ============
from collections import Counter

curve_counter = Counter(all_curves)
print(f"\n所有曲线统计 (共 {len(curve_counter)} 种不同曲线):\n")

# 按出现次数排序
sorted_curves = sorted(curve_counter.items(), key=lambda x: x[1], reverse=True)

for curve, count in sorted_curves:
    percentage = (count / len(well_curves)) * 100
    bar = "█" * int(percentage / 5)
    print(f"{curve:20s} | {count:3d}/{len(well_curves):3d} ({percentage:5.1f}%) {bar}")


In [ ]:
# ============ 3. 找出所有井共有的曲线 ============
print("\n" + "=" * 80)
print(f"\n所有 {len(well_curves)} 口井共有的曲线 (公因数):\n")

if well_curves:
    # 使用集合交集操作找出共有曲线
    common_curves = set(well_curves[list(well_curves.keys())[0]])

    for well_name, curves in well_curves.items():
        common_curves = common_curves.intersection(set(curves))

    common_curves_sorted = sorted(list(common_curves))

    if common_curves_sorted:
        print(f"共有 {len(common_curves_sorted)} 条共同曲线:\n")
        for idx, curve in enumerate(common_curves_sorted, 1):
            print(f"  {idx:2d}. {curve}")
    else:
        print("没有找到所有井都包含的共同曲线!")
else:
    print("没有成功读取任何LAS文件!")

根据测井领域的通用知识，上述曲线可以分为“原始测量曲线”和“解释/计算曲线”。

**原始测量曲线**通常是测井仪器直接测量得到的物理属性。在您列出的曲线中，以下很可能是原始曲线：

- **DEPT**: 深度 (Depth) - 这是所有测井数据的基础，是核心的测量值。
- **GR**: 自然伽马 (Gamma Ray) - 测量地层放射性，最基本的岩性曲线。
- **DEN**: 密度 (Density) - 测量地层体积密度，用于计算孔隙度。
- **DT**: 声波时差 (Delta-T) - 测量声波穿过地层的时间，用于计算孔隙度。
- **CAL**: 井径 (Caliper) - 测量井眼直径。
- **SP**: 自然电位 (Spontaneous Potential) - 测量渗透层和非渗透层之间的电位差。
- **LLD1**: 深侧向电阻率 (Deep Laterolog Resistivity) - 测量地层深部的电阻率。
- **CN**: 补偿中子 (Compensated Neutron) - 测量地层氢指数，用于计算孔隙度。
- **PEFA**: 光电吸收截面指数 (Photoelectric Factor) - 帮助识别岩性，通常与密度仪一起测量。

---

**解释/计算曲线**是基于原始曲线通过公式、模型或机器学习方法计算得出的结果，它们不是直接测量值。列表中的其余大部分属于此类：

- **POR** (孔隙度), **VSH** (泥质含量), **SW** (含水饱和度), **PERM** (渗透率): 这些是经典的储层参数解释结果。
- **FACIES** (岩相), **LITH** (岩性), **FLUIDS** (流体类型): 这些是高级的解释成果，通常是综合多种曲线得出的分类结果。
- **MLP-YC**, **SVM-YC**: 从命名看（MLP/SVM 是机器学习算法），这些极有可能是通过机器学习模型预测出的曲线。
- **GR-NORM**: 标准化伽马，是基于原始 GR 曲线计算处理过的。

因此，如果您想分析最基础的测井数据，应重点关注上面列出的 9 条 **原始测量曲线**。
